In [1]:
import numpy as np
import pandas as pd

In [25]:
val_df = pd.read_csv('../data/val_df.csv')
val_df['FIPS_cty'] = val_df['FIPS_cty'].astype(str)
data = np.load('../data/app_data.npz')
X_test = data['X_test']

In [26]:
val_df.head()

,loc_id,year,yield,yield_pred_og,yield_pred_att,County_State,FIPS_cty
0,0.0,2017.0,55.9,49.729111,54.161774,"adams, illinois",17001
1,1.0,2017.0,42.3,49.615429,46.804111,"alexander, illinois",17003
2,2.0,2017.0,54.5,51.158161,52.772392,"bond, illinois",17005
3,3.0,2017.0,47.4,52.034431,55.832367,"boone, illinois",17007
4,4.0,2017.0,57.1,50.143456,56.247849,"brown, illinois",17009


In [30]:
data = np.load('../data/test_data.npz')
test_data = data['test_data']

In [32]:
test_data[:,0]

array([   0.,    3.,    4.,    5.,    7.,    8.,    9.,   10.,   11.,
         12.,   13.,   14.,   16.,   18.,   19.,   21.,   22.,   23.,
         25.,   26.,   27.,   28.,   29.,   30.,   31.,   32.,   34.,
         35.,   36.,   37.,   38.,   39.,   43.,   44.,   45.,   46.,
         47.,   49.,   50.,   51.,   52.,   53.,   54.,   55.,   56.,
         57.,   64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,
         72.,   73.,   77.,   78.,   79.,   80.,   81.,   82.,   84.,
         87.,   88.,   90.,   92.,   93.,   94.,   95.,   96.,   97.,
         98.,   99.,  100.,  101.,  102.,  103.,  104.,  105.,  106.,
        108.,  109.,  110.,  111.,  112.,  114.,  116.,  117.,  118.,
        119.,  120.,  121.,  124.,  125.,  126.,  127.,  128.,  129.,
        130.,  131.,  132.,  133.,  134.,  135.,  136.,  137.,  138.,
        139.,  140.,  142.,  143.,  147.,  148.,  150.,  152.,  153.,
        155.,  156.,  157.,  158.,  159.,  161.,  164.,  165.,  166.,
        167.,  168.,

In [40]:
import tensorflow as tf

og_model = tf.keras.models.load_model('../models/og_model.keras')
att_model = tf.keras.models.load_model('../models/standard_att_model.keras')


In [48]:

def predict(selected_county):
    loc_id = val_df[val_df['County_State'] == selected_county]['loc_id'].values[0]
    X_sample = test_data[test_data[:, 0] == float(loc_id)]
    if X_sample.shape[0] == 0:
        print('County not found')
        return
    year = X_sample[0, 1]
    y_true = X_sample[0, 2]
    
    X_sample = X_sample[:, 3:]
    #y_pred_og = og_model.predict(X_sample)
    # y_pred_att = att_model.predict(X_sample)
    
    #test_df.loc[len(test_df)] = [loc_id, year, y_true, y_pred_og[0][0], y_pred_att[0][0]]
    print(loc_id, year, y_true, #y_pred_og, 
          X_sample.shape)

In [42]:
selected_county = 'adams, illinois'

In [53]:
def predict_wrapper(model, X, time_steps=5):   
    """
    Wrapper function to handle the input data, format it as dictionary and make predictions.
    """
    X = np.expand_dims(X, axis=-1)
    X_in = {f'w{i}': X[:, 52*i:52*(i+1), :] for i in range(6)}
    X_in.update({f's{i}': X[:, 312+6*i:312+6*(i+1), :] for i in range(11)})
    X_in['p'] = X[:, 378:392, :]
    X_in['avg_yield'] = X[:, - time_steps:, :]
    
    return model.predict(X_in)

In [54]:
try_this = test_data[:2, 3:]

predict_wrapper(og_model,try_this)

1/1 [==============================] - 1s 1s/step


array([[1.2559029],
       [1.5713853]], dtype=float32)